In [1]:
import numpy as np
import matplotlib as plt

import pycuda.driver as cuda_driver
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray

import IPythonMagic
from Timer import Timer

In [2]:
%setup_logging

Python version 3.6.6 (default, Sep 12 2018, 18:26:19) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]


In [3]:
%cuda_context_handler context

Registering context in user workspace
Creating context
PyCUDA version 2018.1.1
CUDA version (9, 1, 0)
Driver version 10000
Using 'Tesla K80' GPU
 => compute capability: (3, 7)
 => memory: 10328 / 11441 MB available
Created context handle <49174640>
Using CUDA cache dir /home/ubuntu/jupyter_notebooks/Fabio/MilanoGPU2018/notebooks/cuda_cache


In [4]:
kernel_src= """



__global__ void shmemReduction(float* output, float* input, int size) {
    //First we stride throug global memory and compute
    //the maximum for every thread
    int gid = blockIdx.x * blockDim.x + threadIdx.x; //blockIdx.x is always zero because we use just one block!
    
    float max_value = -9999999.99; //FIXME: USE PROPER NUMBER
    for (int i = threadIdx.x; i < size; i = i + blockDim.x) { //this gives the nice memory accesso
    
    max_value = fmaxf(max_value, input[i]); 
}

    //Temporary write to memory to check if things work so far
    output[threadIdx.x] = max_value;


    //Store the per-thread maximum in shared memory
    __shared__ float max_shared[128];
    max_shared[threadIdx.x] = max_value;


    //Synchronize so that all thread see the same shared memory
    __syncthreads();


    //Find the maximum in shared memory
    
    //Reduce from 128 to 64 elements
    
    if (threadIdx.x < 64) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 64]);
    }
    
    //since we here have more than one active warp(threadIdx.x > 32)
    //We need to make sure all threads have finished before continuing
    __syncthreads();
    
    //Reduce from 64 to 32 elements
    
    if (threadIdx.x < 32) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 32]);
    }
    
    //Reduce from 32 to 16 elements
    
    if (threadIdx.x < 16) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 16]);
    }
    
    //Reduce from 16 to 8 elements
    if (threadIdx.x < 8) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 8]);
    }
    
    //Reduce from 8 to 4 elements
    if (threadIdx.x < 4) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 4]);
    }
    
    //Reduce from 4 to 2 elements
    if (threadIdx.x < 2) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 2]);
    }
    
    //Reduce from 2 to 1 elements
    if (threadIdx.x < 1) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 1]);
    }
    //Finally write out to output
    
    if (threadIdx.x == 0) {
    output[0] = max_shared[0];
    }
}
"""

kernel_module = cuda_compiler.SourceModule(kernel_src)
kernel_function = kernel_module.get_function("shmemReduction")

In [5]:
n = 256
a = np.random.random((1, n)).astype(np.float32)
print(a)

a_g = GPUArray(a.shape, a.dtype)
a_g.set(a)

num_threads = 128 #each thread the max of 2 numbers
b = np.empty((1, num_threads), dtype=np.float32)

b_g = GPUArray(b.shape, b.dtype)

block_size = (num_threads, 1, 1)
grid_size = (1, 1, 1)

kernel_function(b_g, a_g, np.int32(n), grid=grid_size, block = block_size)

b_g.get(b)

#print(a)
#print(b)
print(np.max(a))

[[0.6881449  0.72943795 0.5265379  0.0485927  0.47310767 0.26235375
  0.30650806 0.81399363 0.62254995 0.11792846 0.19896875 0.53583235
  0.1757224  0.10557467 0.22648853 0.23038836 0.29722062 0.3988975
  0.45321867 0.7799348  0.78821105 0.39444345 0.2839206  0.15808147
  0.71925986 0.12927413 0.19470625 0.11385528 0.40766597 0.6649255
  0.8532714  0.3278626  0.26319018 0.3125056  0.81573933 0.03239956
  0.32090753 0.08855749 0.12553832 0.62899595 0.8279062  0.4880188
  0.06961014 0.6205518  0.9780777  0.5417944  0.07341954 0.16479595
  0.73759574 0.41091383 0.9695671  0.53390026 0.9214707  0.44894105
  0.5145385  0.69200647 0.80482817 0.7688912  0.7003725  0.90557766
  0.36182457 0.23052981 0.21603042 0.35390586 0.86607    0.07634681
  0.54980123 0.4596523  0.71344626 0.15875824 0.76641756 0.3753674
  0.6163961  0.1158151  0.5898228  0.5116801  0.08103568 0.00196694
  0.4242701  0.81932425 0.9008657  0.9965997  0.5672313  0.66611177
  0.8842715  0.51376545 0.33707413 0.29498452 0.9920